In [1]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


2025-03-19 20:11:06.049077: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-19 20:11:06.738108: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-19 20:11:11.303517: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
2.20.0-dev20250319


In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2025-03-19 20:11:32--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.1s    

2025-03-19 20:11:32 (3.27 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2025-03-19 20:11:32--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[============

In [3]:
train_df = pd.read_csv(train_file_path, sep='\t', names=["class", "text"])

test_df = pd.read_csv(test_file_path, sep='\t', names=["class", "text"])

print(train_df)
print(test_df)

     class                                               text
0      ham  ahhhh...just woken up!had a bad dream about u ...
1      ham                           you can never do nothing
2      ham  now u sound like manky scouse boy steve,like! ...
3      ham  mum say we wan to go then go... then she can s...
4      ham  never y lei... i v lazy... got wat? dat day ü ...
...    ...                                                ...
4174   ham  just woke up. yeesh its late. but i didn't fal...
4175   ham  what do u reckon as need 2 arrange transport i...
4176  spam  free entry into our £250 weekly competition ju...
4177  spam  -pls stop bootydelious (32/f) is inviting you ...
4178   ham  tell my  bad character which u dnt lik in me. ...

[4179 rows x 2 columns]
     class                                               text
0      ham  i am in hospital da. . i will return home in e...
1      ham         not much, just some textin'. how bout you?
2      ham  i probably won't eat at all today

In [4]:
train_df["class"] = pd.factorize(train_df["class"])[0]
test_df["class"] = pd.factorize(test_df["class"])[0]

print(train_df)
print(test_df)

      class                                               text
0         0  ahhhh...just woken up!had a bad dream about u ...
1         0                           you can never do nothing
2         0  now u sound like manky scouse boy steve,like! ...
3         0  mum say we wan to go then go... then she can s...
4         0  never y lei... i v lazy... got wat? dat day ü ...
...     ...                                                ...
4174      0  just woke up. yeesh its late. but i didn't fal...
4175      0  what do u reckon as need 2 arrange transport i...
4176      1  free entry into our £250 weekly competition ju...
4177      1  -pls stop bootydelious (32/f) is inviting you ...
4178      0  tell my  bad character which u dnt lik in me. ...

[4179 rows x 2 columns]
      class                                               text
0         0  i am in hospital da. . i will return home in e...
1         0         not much, just some textin'. how bout you?
2         0  i probably won't 

In [5]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (train_df["text"].values, train_df["class"].values)
)
test_ds = tf.data.Dataset.from_tensor_slices(
    (test_df["text"].values, test_df["class"].values)
)

I0000 00:00:1742425902.961077    6943 gpu_device.cc:2018] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3542 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


In [6]:
BUFFER_SIZE = 100
BATCH_SIZE = 32

train_ds = train_ds.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [7]:
df = pd.concat([train_df, test_df], ignore_index=True)

vocab = set(word for text in df['text'].dropna() for word in text.split())

# Vocabulary size
max_vocab_size = len(vocab)

print("Maximum vocabulary size:", max_vocab_size)

Maximum vocabulary size: 13484


In [8]:
MAXLEN = 1000

vec = keras.layers.TextVectorization(
    output_mode='int',
    max_tokens=max_vocab_size,
    output_sequence_length=MAXLEN,
)

vec.adapt(train_ds.map(lambda text, label: text))

2025-03-19 20:11:50.662114: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [26]:
model = tf.keras.Sequential([
    vec,
    keras.layers.Embedding(max_vocab_size, 64),
    keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(1)
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss='binary_crossentropy',
    metrics=['acc']
)

model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ text_vectorization              │ (None, 1000)           │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_4 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_5 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [27]:
history = model.fit(
    train_ds,
    validation_data=test_ds,
    validation_steps=30,
    epochs=5,
)

Epoch 1/5
131/131 ━━━━━━━━━━━━━━━━━━━━ 68s 471ms/step - acc: 0.8707 - loss: 0.5107 - val_acc: 0.8604 - val_loss: 0.3301
Epoch 2/5
131/131 ━━━━━━━━━━━━━━━━━━━━ 56s 424ms/step - acc: 0.8769 - loss: 0.2612 - val_acc: 0.9406 - val_loss: 0.2085
Epoch 3/5
131/131 ━━━━━━━━━━━━━━━━━━━━ 56s 428ms/step - acc: 0.9634 - loss: 0.1312 - val_acc: 0.9740 - val_loss: 0.1715
Epoch 4/5
131/131 ━━━━━━━━━━━━━━━━━━━━ 57s 432ms/step - acc: 0.9850 - loss: 0.0758 - val_acc: 0.9802 - val_loss: 0.1881
Epoch 5/5
131/131 ━━━━━━━━━━━━━━━━━━━━ 56s 425ms/step - acc: 0.9927 - loss: 0.0479 - val_acc: 0.9802 - val_loss: 0.1857


In [28]:
test_loss, test_acc = model.evaluate(test_ds)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

44/44 ━━━━━━━━━━━━━━━━━━━━ 10s 232ms/step - acc: 0.9819 - loss: 0.1569
Test Loss: 0.17112551629543304
Test Accuracy: 0.9798850417137146


In [29]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    
    sample_tensor = tf.convert_to_tensor([pred_text])

    prediction = model.predict(sample_tensor)[0][0]

    prob = tf.nn.sigmoid(prediction).numpy()
    
    print(prob, "ham" if prob < 0.5 else "spam")

    return [prob.item(), "ham" if prob < 0.5 else "spam"]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
0.40119064 ham
[0.4011906385421753, 'ham']


In [30]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 455ms/step
0.40119064 ham
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step
0.5191843 spam
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 225ms/step
0.39498904 ham
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 254ms/step
0.75418544 spam
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step
0.77936256 spam
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step
0.39240456 ham
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 227ms/step
0.41823515 ham
You passed the challenge. Great job!
